# Markdown as valid code

Recently, some posts have been introduced as changes to `pidgin` which provides some literate computing extensions for the Jupyter notebook.  It effectively combines the following modules we created.

In [1]:
    from . import (
        __String_Node_Transformer, 
        __Markdown_code_cells, 
        __Jinja2_Templating_Transformer)

In [2]:
    from CommonMark import Parser
    from CommonMark.render.renderer import Renderer
    from textwrap import indent, dedent
    from importnb import Execute
    from collections import UserList
    from abc import abstractmethod, ABCMeta
    try:
        from IPython.display import display, Markdown, HTML
        from IPython.core.inputtransformer import InputTransformer
        from IPython import get_ipython
    except:
        class InputTrgansformer:
            def __init__(self, *args, **kwargs): ...
            def transform_cell(self, str): return dedent(str)
            
        def get_ipython(): ...

In [91]:
    %%html
    <style>
    .inspectable {
        display: inline;
    }
    .inspectable code {
        display: inline-block;
        text-decoration: underline;
    }
    .inspectable:hover code{
        text-decoration: overline;
    }
    .inspectable .inspect {
        display: none;
        border-style: solid;
        border-width: 5px;
    }
    .inspectable:hover .inspect {
        display: block;
    }
    </style>

In [92]:
    import ast

In [93]:
    from CommonMark import Parser
    from CommonMark.render.html import HtmlRenderer

    from IPython.core.oinspect import Inspector

    inspector = Inspector(scheme='NoColor')

    from IPython.utils.capture import capture_output

    class IMarkdownRenderer(HtmlRenderer):
        def code(self, node, entering):
            with capture_output(stderr=False, stdout=False) as out:
                module = ast.parse(node.literal)
                for object in module.body:
                    if object is module.body[-1] and isinstance(module.body[-1], ast.Expr):
                        code = compile(ast.Expression(object.value), '<inspected>', 'eval')
                    else:
                        code = compile(ast.Module([object]), '<inspected>', 'exec')
                    object = eval(code, get_ipython().user_ns)
                if object is not None:
                    display(object)

            if out.outputs and 'text/html' in  out.outputs[0].data:
                overlay = out.outputs[0].data['text/html']
            elif object is not None:
                overlay = inspector._get_info(object, node.literal)['text/html']
            else: 
                overlay = ""

            begin, end = self.buf.rsplit('>', 1)

            self.buf = begin + ' style="display: inline-block">' + end
            self.tag('div', (('class', 'inspectable'),))
            super().code(node, entering)
            if overlay:
                self.tag('div', (('class', 'inspect'),))
                self.lit(overlay), self.tag('br', selfclosing=True), self.tag('/div')
            self.tag('/div')
            return

In [94]:
def show_interactive(str):
    return display(HTML(IMarkdownRenderer().render(Parser().parse(str))))

In [95]:
    from pidgin.markdown import MarkdownTransformer, MarkdownImporter, render

In [96]:
    class IMarkdownTransformer(MarkdownTransformer):
        def __call__(self, str): 
            return render(str, display=show_interactive)

    class IMarkdownImporter(MarkdownImporter):
        extensions = '.i.md.ipynb',
        def format(self, str): 
            return super().format(render(str, renderer=show_interactive))

In [97]:
    def load_ipython_extension(ip=None):
        ip = ip or get_ipython()
        ip.input_transformer_manager.physical_line_transforms = [
            IMarkdownTransformer()] + [
            object for object in ip.input_transformer_manager.physical_line_transforms
            if not isinstance(object, (MarkdownTransformer, IMarkdownTransformer))]
            
    def unload_ipython_extension(ip=None):
        ip = ip or get_ipython()
        ip.input_transformer_manager.physical_line_transforms = list(
            object for object in ip.input_transformer_manager.physical_line_transforms
            if not isinstance(object, (MarkdownTransformer, IMarkdownTransformer))
        )

In [98]:
    if __name__ == '__main__':
        load_ipython_extension()

In [99]:
    import pandas, pidgin

In [100]:
# What is special now?

The `pidgin.markdown` module allow code cells to be markdown and the block code is concatenated into a single code execution.
The inline code elements are meaningless by these conventions.

> It has been nagging me for a while what to do with inline code in Markdown.  

## Interactive Markdown

In interactive markdown, inline code cells have augmented views.  Like the author would share what 
the current state of the `globals().keys()` are.  If the notebook will restart and run all then these
outputs can be encoded into documents.

Usually there is and data.  When there is data with `import pandas as pd;pd`
and create a `pd.DataFrame`.  The quickest way to do so is with the `pd.util.testing` module; 
`df = pd.util.testing.makeDataFrame(); df` and has the following properties. 

    df.describe()

,A,B,C,D
SjfLAAiqDh,-0.286437,1.073016,-0.070179,-0.321905
pOH6Mz7sRT,-0.285672,0.285390,-1.155438,-0.376187
hucinWSYR8,0.127173,-1.470252,-0.074350,-0.307167
uUykwVUIxD,-0.392699,-0.431904,-0.089746,0.839722
wDIdaywizC,0.078331,-0.295204,0.653159,1.624445
aZytWi3qYf,0.097397,0.280280,0.716109,2.349472
9XOJQwPKkU,-0.654928,-1.724795,0.689734,-0.344019
6oF6HpdU2L,0.300741,0.750027,0.232418,2.330587
WtVKRO7QuL,-1.077829,0.925302,-1.107383,-1.439161
nQbRSVcefO,0.384150,0.730625,1.745764,0.279001


,A,B,C,D
count,30.000000,30.000000,30.000000,30.000000
mean,0.164658,0.205865,0.152672,0.218687
std,1.101898,0.871088,0.731742,1.044663
min,-1.834202,-1.724795,-1.155438,-1.439161
25%,-0.573153,-0.290726,-0.323388,-0.436560
50%,0.112285,0.278372,0.016640,0.285380
75%,0.908357,0.745177,0.650977,0.892411
max,2.897978,2.078704,1.745764,2.349472
